In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings("ignore")

In [2]:
signal_data = pd.read_csv('signal.csv')
option_data = pd.read_csv('nifty_opt_data.csv')

In [3]:
def get_user_inputs():
    # A dictionary containing the user inputs for the trading strategy
    user_inputs = {
        'start_date': '2023-12-28',      # Start date in 'YYYY-MM-DD' format
        'end_date': '2024-02-29',        # End date in 'YYYY-MM-DD' format
        'timeframe': '1Minute',          # Timeframe for the strategy
        #'ticker': 'NIFTY',               # Stock ticker symbol
        'strategy_type': 'Long_Straddle', # Strategy type (use snake_case for variable names)
        'strike_step': 50,                # Strike step for the strategy
        'strike_type': 'ATM',           # Strike type
        'entry_time': '10:15',           # Entry time in 'HH:MM' format
        'exit_time': '15:15',            # Exit time in 'HH:MM' format
        'tg_leg1': 0.2,                  # Target for leg 1
        'tg_leg2': 0.2,                  # Target for leg 2
        'sl_leg1': 0.1,                 # Stop loss for leg 1
        'sl_leg2': 0.1,                 # Stop loss for leg 2
    }

    return user_inputs

In [4]:
def filtered_option_data(option_data, user_inputs):
    # Filter the option data based on the user inputs
    option_data['date'] = pd.to_datetime(option_data['timestamp']).dt.date
    option_data['time'] = pd.to_datetime(option_data['timestamp']).dt.time

    # Convert start and end dates to Timestamps
    start_date = pd.to_datetime(user_inputs['start_date'], format='%Y-%m-%d').date()
    end_date = pd.to_datetime(user_inputs['end_date'], format='%Y-%m-%d').date()

    # Filter the option data based on the start and end dates
    option_data = option_data[(option_data['date'] >= start_date) & (option_data['date'] <= end_date)]
    option_data = option_data[option_data['time'] >= pd.to_datetime(user_inputs['entry_time']).time()]
    option_data['strike'] = option_data['strike'].astype(int).astype(str) + option_data['option_type']

    option_data.reset_index(drop=True, inplace=True)
    
    return option_data

In [5]:
# Example usage of the function
user_inputs = get_user_inputs()
option_data = filtered_option_data(option_data, user_inputs)
option_data

,timestamp,open,high,low,close,strike,expiry,option_type,date,time
0,2023-12-28 10:15:00,0.20,0.20,0.20,0.20,11000PE,2023-12-28,PE,2023-12-28,10:15:00
1,2023-12-28 10:15:00,0.30,0.30,0.30,0.30,14000PE,2023-12-28,PE,2023-12-28,10:15:00
2,2023-12-28 10:15:00,0.25,0.25,0.25,0.25,15000PE,2023-12-28,PE,2023-12-28,10:15:00
3,2023-12-28 10:15:00,4715.70,4719.35,4715.70,4719.35,17000CE,2023-12-28,CE,2023-12-28,10:15:00
4,2023-12-28 10:15:00,0.25,0.30,0.25,0.30,17000PE,2023-12-28,PE,2023-12-28,10:15:00
...,...,...,...,...,...,...,...,...,...,...
1693108,2024-02-29 15:20:00,316.95,316.95,316.95,316.95,22300PE,2024-02-29,PE,2024-02-29,15:20:00
1693109,2024-02-29 15:20:00,0.15,0.15,0.15,0.15,22350CE,2024-02-29,CE,2024-02-29,15:20:00
1693110,2024-02-29 15:20:00,0.05,0.05,0.05,0.05,22400CE,2024-02-29,CE,2024-02-29,15:20:00
1693111,2024-02-29 15:20:00,0.05,0.05,0.05,0.05,22450CE,2024-02-29,CE,2024-02-29,15:20:00


In [6]:
unique_dates = option_data['expiry'].unique()
expiry_dates = pd.to_datetime(unique_dates).date

In [7]:
# Function to preprocess the signal data based on the user inputs
def signal_data_preprocessing(signal_data, user_inputs):
    signal_data['date'] = pd.to_datetime(signal_data['timestamp']).dt.date
    signal_data['time'] = pd.to_datetime(signal_data['timestamp']).dt.time

    # Create the expiry column based on the date
    def assign_expiry(date):
        # Check each expiry date to find the correct one
        for i in range(len(expiry_dates)):
            if date <= expiry_dates[i]:
                return expiry_dates[i]
            
    signal_data['expiry'] = signal_data['date'].apply(assign_expiry)

    # Convert start and end dates to Timestamps
    start_date = pd.to_datetime(user_inputs['start_date'], format='%Y-%m-%d').date()
    end_date = pd.to_datetime(user_inputs['end_date'], format='%Y-%m-%d').date()

    # Filter the signal data based on the start and end dates
    signal_data = signal_data[(signal_data['date'] >= start_date) & (signal_data['date'] <= end_date)]
    signal_data = signal_data[signal_data['time'] >= pd.to_datetime(user_inputs['entry_time']).time()]
    signal_data = signal_data[signal_data['time'] <= pd.to_datetime(user_inputs['exit_time']).time()]

    # Get the strike movement (OTM) value from user inputs
    if user_inputs['strike_type'].startswith('OTM') or user_inputs['strike_type'].startswith('ITM'):
        value = int(user_inputs['strike_type'][-1])
    strike_step = user_inputs['strike_step']  # Assuming this is the step between strikes (e.g., 50, 100)

    # Check strike type from the user inputs and create 'leg1' and 'leg2' columns accordingly
    if user_inputs['strike_type'] == 'ATM':
        signal_data['leg1_strike'] = signal_data['strike']  # At The Money
        signal_data['leg2_strike'] = signal_data['strike']  # At The Money
    
    elif user_inputs['strike_type'].startswith('OTM'):
        # Adjust leg1 and leg2 strikes based on the OTM value and step
        signal_data['leg1_strike'] = signal_data['strike'] + (strike_step * value)  # Move OTM for CE (Call)
        signal_data['leg2_strike'] = signal_data['strike'] - (strike_step * value)  # Move OTM for PE (Put)

    elif user_inputs['strike_type'].startswith('ITM'):
        # Adjust leg1 and leg2 strikes based on the ITM value and step
        signal_data['leg1_strike'] = signal_data['strike'] - (strike_step * value)
        signal_data['leg2_strike'] = signal_data['strike'] + (strike_step * value)
    
    # Create 'leg1_strike' and 'leg2_strike' columns as strings with suffixes
    # Ensure leg strikes are formatted correctly
    signal_data['leg1_strike'] = (signal_data['leg1_strike'].astype(int).astype(str) + 'CE')
    signal_data['leg2_strike'] = (signal_data['leg2_strike'].astype(int).astype(str) + 'PE')


    signal_data.reset_index(drop=True, inplace=True)
    
    return signal_data


In [8]:

signal_data = signal_data_preprocessing(signal_data, user_inputs)
signal_data

,timestamp,open,high,low,close,signal,strike,date,time,expiry,leg1_strike,leg2_strike
0,2023-12-28 10:15:00,21697.50,21698.15,21689.80,21691.50,0,21700.0,2023-12-28,10:15:00,2023-12-28,21700CE,21700PE
1,2023-12-28 10:16:00,21692.25,21705.10,21692.20,21700.20,0,21700.0,2023-12-28,10:16:00,2023-12-28,21700CE,21700PE
2,2023-12-28 10:17:00,21700.30,21706.20,21697.75,21704.15,0,21700.0,2023-12-28,10:17:00,2023-12-28,21700CE,21700PE
3,2023-12-28 10:18:00,21704.85,21712.90,21703.20,21708.40,1,21700.0,2023-12-28,10:18:00,2023-12-28,21700CE,21700PE
4,2023-12-28 10:19:00,21709.10,21710.55,21703.70,21708.70,0,21700.0,2023-12-28,10:19:00,2023-12-28,21700CE,21700PE
...,...,...,...,...,...,...,...,...,...,...,...,...
12938,2024-02-27 15:11:00,22204.10,22208.95,22201.95,22208.70,0,22200.0,2024-02-27,15:11:00,2024-02-29,22200CE,22200PE
12939,2024-02-27 15:12:00,22209.55,22210.00,22204.40,22205.90,0,22200.0,2024-02-27,15:12:00,2024-02-29,22200CE,22200PE
12940,2024-02-27 15:13:00,22206.55,22209.45,22204.75,22208.90,0,22200.0,2024-02-27,15:13:00,2024-02-29,22200CE,22200PE
12941,2024-02-27 15:14:00,22210.00,22211.15,22202.95,22203.90,0,22200.0,2024-02-27,15:14:00,2024-02-29,22200CE,22200PE


In [9]:
sl_leg1 = user_inputs['sl_leg1']
sl_leg2 = user_inputs['sl_leg2']
tg_leg1 = user_inputs['tg_leg1']
tg_leg2 = user_inputs['tg_leg2']

# Convert time to datetime time format
option_data['time'] = pd.to_datetime(option_data['time'], format='%H:%M:%S').dt.time
option_data['date'] = pd.to_datetime(option_data['date']).dt.date

# Convert expiry to date format, stripping time part
option_data['expiry'] = pd.to_datetime(option_data['expiry']).dt.date

option_data['strike'] = option_data['strike'].str.strip()

# Set the index for option_data
filtered_option_data = option_data.set_index(['date', 'time', 'strike', 'expiry'])

if user_inputs['strategy_type'] == 'Long_Straddle':
    signal = 1
elif user_inputs['strategy_type'] == 'Short_Straddle':
    signal = -1

# Initialize a list to store rows before creating DataFrame
tradebook_rows = []

# Loop through signal data to process trades
for i in range(len(signal_data)):
    if signal_data['signal'][i] == signal:
        entry_date = pd.to_datetime(signal_data['date'][i], format='%Y-%m-%d').date()  # Extract date only
        entry_time = pd.to_datetime(signal_data['time'][i], format='%H:%M:%S').time()
        expiry = pd.to_datetime(signal_data['expiry'][i]).date()  # Extract date only
        
        # Process Leg 1
        leg1_strike = signal_data['leg1_strike'][i]
        OpenPosDict = {
            'Leg': 'Leg1',
            'Strike_Type': leg1_strike,
            'Expiry': expiry,
            'Entry_Date': entry_date,
            'Entry_Time': entry_time,
            'Exit_Date': None,
            'Exit_Time': None,
            'Exit_Price': None,
            'Remarks': None
        }
        
        try:
            entry_price = filtered_option_data.loc[(entry_date, entry_time, leg1_strike, expiry), 'open']
            OpenPosDict['Entry_Price'] = entry_price
            OpenPosDict['Stoploss'] = entry_price - (entry_price * sl_leg1)
            OpenPosDict['Takeprofit'] = entry_price + (entry_price * tg_leg1)
        except KeyError:
            OpenPosDict['Entry_Price'] = None
            OpenPosDict['Stoploss'] = None
            OpenPosDict['Takeprofit'] = None
        
        tradebook_rows.append(OpenPosDict.copy())
        
        # Process Leg 2
        leg2_strike = signal_data['leg2_strike'][i]
        OpenPosDict['Leg'] = 'Leg2'
        OpenPosDict['Strike_Type'] = leg2_strike
        
        try:
            entry_price = filtered_option_data.loc[(entry_date, entry_time, leg2_strike, expiry), 'open']
            OpenPosDict['Entry_Price'] = entry_price
            OpenPosDict['Stoploss'] = entry_price - (entry_price * sl_leg2)
            OpenPosDict['Takeprofit'] = entry_price + (entry_price * tg_leg2)
        except KeyError:
            OpenPosDict['Entry_Price'] = None
            OpenPosDict['Stoploss'] = None
            OpenPosDict['Takeprofit'] = None
        
        tradebook_rows.append(OpenPosDict.copy())

# Define the column order explicitly
column_order = [
    'Leg', 'Strike_Type', 'Expiry', 'Entry_Date', 'Entry_Time', 
    'Entry_Price', 'Stoploss', 'Takeprofit', 
    'Exit_Date', 'Exit_Time', 'Exit_Price', 'Remarks'
]

# Create the DataFrame from the list of rows and specify the column order
tradebook = pd.DataFrame(tradebook_rows, columns=column_order)
tradebook.dropna(subset=['Entry_Price'], inplace=True)


In [10]:
tradebook

,Leg,Strike_Type,Expiry,Entry_Date,Entry_Time,Entry_Price,Stoploss,Takeprofit,Exit_Date,Exit_Time,Exit_Price,Remarks
0,Leg1,21700CE,2023-12-28,2023-12-28,10:18:00,61.80,55.620,74.16,None,None,None,None
1,Leg2,21700PE,2023-12-28,2023-12-28,10:18:00,42.90,38.610,51.48,None,None,None,None
2,Leg1,21700CE,2023-12-28,2023-12-28,10:24:00,62.25,56.025,74.70,None,None,None,None
3,Leg2,21700PE,2023-12-28,2023-12-28,10:24:00,40.60,36.540,48.72,None,None,None,None
4,Leg1,21700CE,2023-12-28,2023-12-28,10:28:00,59.25,53.325,71.10,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
1241,Leg2,22150PE,2024-02-29,2024-02-27,14:05:00,104.05,93.645,124.86,None,None,None,None
1242,Leg1,22150CE,2024-02-29,2024-02-27,14:34:00,119.75,107.775,143.70,None,None,None,None
1243,Leg2,22150PE,2024-02-29,2024-02-27,14:34:00,96.70,87.030,116.04,None,None,None,None
1244,Leg1,22200CE,2024-02-29,2024-02-27,14:51:00,96.70,87.030,116.04,None,None,None,None


In [11]:
######## Check for exit conditions ########
# Ensure all required columns are stripped of whitespace
option_data['strike'] = option_data['strike'].str.strip()
tradebook['Strike_Type'] = tradebook['Strike_Type'].str.strip()

# Convert date and time columns to appropriate formats
option_data['date'] = pd.to_datetime(option_data['date'], format='%Y-%m-%d').dt.date
tradebook['Entry_Date'] = pd.to_datetime(tradebook['Entry_Date'], format='%Y-%m-%d').dt.date
tradebook['Entry_Time'] = pd.to_datetime(tradebook['Entry_Time'], format='%H:%M:%S').dt.time
option_data['time'] = pd.to_datetime(option_data['time'], format='%H:%M:%S').dt.time
tradebook['Expiry'] = pd.to_datetime(tradebook['Expiry'], format='%Y-%m-%d').dt.date
option_data['expiry'] = pd.to_datetime(option_data['expiry'], format='%Y-%m-%d').dt.date

# Parse exit_time from user inputs
exit_time = pd.to_datetime(user_inputs['exit_time'], format='%H:%M').time()

# Reset the index of tradebook to ensure sequential indexing
tradebook = tradebook.reset_index(drop=True)

# Loop through the tradebook to process exits
for i in range(len(tradebook)):
    strike = tradebook['Strike_Type'][i]
    entry_date = tradebook['Entry_Date'][i]
    entry_time = tradebook['Entry_Time'][i]
    expiry = tradebook['Expiry'][i]

    # Filter option data for the specific trade
    filtered_data = option_data.loc[(option_data['date'] == entry_date) & (option_data['time'] >= entry_time) & 
                                    (option_data['time'] <= exit_time) & (option_data['strike'] == strike) & 
                                    (option_data['expiry'] == expiry)]
    

    # Reset index of filtered_data to avoid KeyErrors
    filtered_data = filtered_data.reset_index(drop=True)

    # Check if filtered_data is empty
    if filtered_data.empty:
        print(f"No matching data found for trade {i}")
        continue

    # Initialize an open position dictionary to store exit details for this trade
    openpos_dict = {"Exit_date": None, "Exit_time": None, "Exit_price": None, "Remarks": None}

    for j in range(len(filtered_data)):
        if filtered_data['high'][j] >= tradebook['Takeprofit'][i]:
            openpos_dict['Exit_date'] = filtered_data['date'][j]
            openpos_dict['Exit_time'] = filtered_data['time'][j]
            openpos_dict['Exit_price'] = tradebook['Takeprofit'][i]
            openpos_dict['Remarks'] = 'Takeprofit'
            break
        elif filtered_data['low'][j] <= tradebook['Stoploss'][i]:
            openpos_dict['Exit_date'] = filtered_data['date'][j]
            openpos_dict['Exit_time'] = filtered_data['time'][j]
            openpos_dict['Exit_price'] = tradebook['Stoploss'][i]
            openpos_dict['Remarks'] = 'Stoploss'
            break
        elif filtered_data['time'][j] == exit_time:
            openpos_dict['Exit_date'] = filtered_data['date'][j]
            openpos_dict['Exit_time'] = filtered_data['time'][j]
            openpos_dict['Exit_price'] = filtered_data['close'][j]
            openpos_dict['Remarks'] = 'Exit Time'
            break

    # Update the tradebook with exit details
    tradebook.loc[i, 'Exit_Date'] = openpos_dict['Exit_date']
    tradebook.loc[i, 'Exit_Time'] = openpos_dict['Exit_time']
    tradebook.loc[i, 'Exit_Price'] = openpos_dict['Exit_price']
    tradebook.loc[i, 'Remarks'] = openpos_dict['Remarks']


In [12]:
tradebook

In [ ]:
#tradebook.to_csv('tradebook.csv', index=False)